In [1]:
import syft as sy
sy.LOG_FILE = "syft_ds.log"
_ = sy.logger.add(sy.LOG_FILE, enqueue=True, colorize=False, diagnose=True, backtrace=True, level="TRACE")

In [2]:
duet = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ joining duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently 
♫♫♫ > in alpha. Do not use this to protect real-world data.
♫♫♫ >
♫♫♫ > Punching through firewall to OpenGrid Network Node at network_url: 
♫♫♫ > http://ec2-18-216-8-163.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... DONE!

♫♫♫ > Duet Client ID: a9565988e5b9fba9dbbf055fb97a8ae8

♫♫♫ > STEP 1: Send the Duet Client ID to your duet partner!

♫♫♫ > ...waiting for partner to connect...
♫♫♫ > ...using a running event loop...

♫♫♫ > CONNECTED!


In [3]:
# lets get some references to our data owners Duet torch and torchvision
torch = duet.torch
torchvision = duet.torchvision

# these are the same as the original mnist example
transforms = torchvision.transforms
datasets = torchvision.datasets
nn = torch.nn
F = torch.nn.functional
optim = torch.optim
StepLR = torch.optim.lr_scheduler.StepLR

In [4]:
import torch as th
import torchvision as tv

In [5]:
# Training settings from original MNIST example command line args
args = {
    "batch_size": 64,
    "test_batch_size": 1000,
    "epochs": 14,
    "lr": 1.0,
    "gamma": 0.7,
    "no_cuda": False,
    "dry_run": False,
    "seed": 42, # the meaning of life
    "log_interval": 10,
    "save_model": False,
}

In [6]:
model = nn.Sequential(
    nn.Conv2d(1, 32, 3, 1),
    nn.ReLU(),
    nn.Conv2d(32, 64, 3, 1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout2d(0.25),
    nn.Flatten(),
    nn.Linear(9216, 128),
#     nn.ReLU(),
#     nn.Dropout2d(0.5),
#     nn.Linear(128, 10),
    nn.LogSoftmax(dim=1),
)

In [7]:
model.cpu()

In [8]:
# lets get our parameters for optimization
# params_list required for remote list concatenation
params = model.parameters(params_list=duet.syft.lib.python.List())
print(params, type(params))

<syft.proxy.syft.lib.python.ListPointer object at 0x7f0bda698ca0> <class 'syft.proxy.syft.lib.python.ListPointer'>


In [9]:
optimizer = optim.Adadelta(params, lr=args["lr"])
print(optimizer, type(optimizer))

<syft.proxy.torch.optim.AdadeltaPointer object at 0x7f0bda6ba820> <class 'syft.proxy.torch.optim.AdadeltaPointer'>


In [10]:
scheduler = StepLR(optimizer, step_size=1, gamma=args["gamma"])
print(scheduler, type(scheduler))

<syft.proxy.torch.optim.lr_scheduler.StepLRPointer object at 0x7f0bda6ab6a0> <class 'syft.proxy.torch.optim.lr_scheduler.StepLRPointer'>


In [11]:
from time import sleep
# now can define a simple training loop very similar to the original PyTorch MNIST example
@sy.logger.catch
def train(args, model, train_loader, optimizer, epoch, train_data_length):
    train_batches = round((train_data_length / args["batch_size"]) + 0.5)
    model.train()

    for batch_idx, data in enumerate(train_loader):
        
        data_ptr, target_ptr = data[0], data[1]
        optimizer.zero_grad()
        output = model(data_ptr)
        loss = F.nll_loss(output, target_ptr)
        loss.backward()
        optimizer.step()

        if batch_idx >= 15:
            print("batch_idx >= train_batches, breaking")
            break

In [12]:
# we need some transforms for our MNIST data set
transform_1 = torchvision.transforms.ToTensor()  # this converts PIL images to Tensors
transform_2 = torchvision.transforms.Normalize(0.1307, 0.3081)  # this normalizes the dataset

remote_list = duet.syft.lib.python.List()
remote_list.append(transform_1)
remote_list.append(transform_2)

# compose our transforms
transforms = torchvision.transforms.Compose(remote_list)

# The DO has kindly let us initialise a DataLoader for their training set
train_kwargs = {
    "batch_size": args["batch_size"],
}
train_data_ptr = torchvision.datasets.MNIST('../data', train=True, download=True, transform=transforms)
train_loader_ptr = torch.utils.data.DataLoader(train_data_ptr,**train_kwargs)

In [13]:
def get_train_length(train_data_ptr):
    train_length_ptr = train_data_ptr.__len__()
    train_data_length = train_length_ptr.get(
        request_block=True,
        name="train_size",
        reason="To write the training loop"
    )
    return train_data_length

try:
    if train_data_length is None:
        train_data_length = get_train_length(train_data_ptr)
except NameError:
        train_data_length = get_train_length(train_data_ptr)

print(f"Training Dataset size is: {train_data_length}")


> Waiting for Blocking Request
train_size: To write the training loop.
<UID:d753f607-4e1f-4bbc-befd-ad136670e0f8>
.......
> Blocking Request ACCEPTED
Training Dataset size is: 60000


In [14]:
%%time
# sequential CPU times: user 39.2 s, sys: 390 ms, total: 39.6s per epoch
# sy.module CPU times: user 1min 28s, sys: 645 ms, total: 1min 28s per epoch
# vanilla class CPU times: user 3min 5s, sys: 3.01 s, total: 3min 8s per epoch
args["epochs"] = 1
sy.logger.trace("Start Training")
for epoch in range(1, args["epochs"] + 1):
    train(args, model, train_loader_ptr, optimizer, epoch, train_data_length)
    scheduler.step()
sy.logger.trace("Finish Training")

batch_idx >= train_batches, breaking
CPU times: user 1.15 s, sys: 44.9 ms, total: 1.19 s
Wall time: 1.15 s


In [15]:
param_list_ptr = duet.syft.lib.python.List(model.parameters())

In [ ]:
param_list = param_list_ptr.get(
    request_block=True,
    name="Param list",
    reason="Param list to see if it generalizes on my local test set"
)


> Waiting for Blocking Request
Param list: Param list to see if it generalizes on my local test set.
<UID:faf45286-3bea-4e61-acc4-01e67e1426a5>
..

Exception in callback Transaction.__retry()
handle: <TimerHandle when=20149.235858258 Transaction.__retry()>
Traceback (most recent call last):
  File "/home/tudor/anaconda3/envs/syft_0.3.0/lib/python3.8/site-packages/nest_asyncio.py", line 194, in run
    ctx.run(self._callback, *self._args)
  File "/home/tudor/anaconda3/envs/syft_0.3.0/lib/python3.8/site-packages/aioice/stun.py", line 299, in __retry
    self.__future.set_exception(TransactionTimeout())
  File "/home/tudor/anaconda3/envs/syft_0.3.0/lib/python3.8/asyncio/futures.py", line 246, in set_exception
    raise exceptions.InvalidStateError(f'{self._state}: {self!r}')
asyncio.exceptions.InvalidStateError: FINISHED: <Future finished result=(Message(messa...\xab\xea\x97'), ('192.168.20.207', 40818))>


...
> Blocking Request ACCEPTED


In [ ]:
print(param_list)

In [ ]:
import torch as local_torch
import torchvision as local_torchvision
from torchvision import transforms as local_transforms
from torchvision import datasets as local_datasets
from torch import nn as local_nn
from torch.nn import functional as local_F
from torch import optim as local_optim
from torch.optim.lr_scheduler import StepLR as local_StepLR


In [ ]:
model = local_nn.Sequential(
    local_nn.Conv2d(1, 32, 3, 1),
    local_nn.ReLU(),
    local_nn.Conv2d(32, 64, 3, 1),
    local_nn.ReLU(),
    local_nn.MaxPool2d(2),
    local_nn.Dropout2d(0.25),
    local_nn.Flatten(),
    local_nn.Linear(400, 128),
    local_nn.ReLU(),
    local_nn.Dropout2d(0.5),
    local_nn.Linear(128, 10),
    local_nn.LogSoftmax(dim=1),
)

In [ ]:
local_params = list(model.parameters())
for i in range(len(local_params)):
    local_params[i].data[:] = param_list[i].data[:]

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with local_torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += local_F.nll_loss(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )

transform = local_transforms.Compose(
    [local_transforms.ToTensor(), local_transforms.Normalize((0.1307,), (0.3081,))]
)
    
dataset2 = local_datasets.MNIST("../data", train=False, transform=transform)
test_loader = local_torch.utils.data.DataLoader(dataset2, batch_size=64)

In [ ]:
test(model, "cpu", test_loader)